<a href="https://colab.research.google.com/github/mitosagi/puzzdra-nnsolver/blob/master/puzz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 初期化

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r /content/drive/MyDrive/User/python/puzzdra-nnsolver /content/puzzdra-nnsolver
%cd /content/puzzdra-nnsolver
!pip install --log=pip_log -e .
!pip install git+https://github.com/DLR-RM/stable-baselines3

/content/puzzdra-nnsolver
Obtaining file:///content/puzzdra-nnsolver
  Running setup.py develop for Puzzpy
163663
334311
323365
136624
145342
[[1, 3, 6, 6, 6, 3], [3, 3, 4, 3, 1, 1], [3, 2, 3, 3, 6, 5], [1, 3, 6, 6, 2, 4], [1, 4, 5, 3, 4, 2]]
[[0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
[[127, 6, 3, 6, 6, 3], [3, 3, 4, 3, 1, 1], [3, 2, 3, 3, 6, 5], [1, 3, 6, 6, 2, 4], [1, 4, 5, 3, 4, 2]]
[[0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
[[1, 6, 4, 6, 6, 3], [3, 3, 3, 3, 1, 1], [3, 2, 3, 3, 6, 5], [1, 3, 6, 6, 2, 4], [1, 4, 5, 3, 4, 2]]
[[0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
[[1, 6, 6, 3, 6, 3], [3, 3, 4, 3, 1, 1], [3, 2, 3, 3, 6, 5], [1, 3, 6, 6, 2, 4], [1, 4, 5, 3, 4, 2]]
[[0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
10
453132
566644
565226
525664
462244
<class 'in

## 実際の処理

In [145]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.type_aliases import Schedule
from stable_baselines3.common.preprocessing import get_flattened_obs_dim, is_image_space
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union
import torch as th
from torch import nn

class NatureCNN(BaseFeaturesExtractor):
    """
    CNN from DQN nature paper:
        Mnih, Volodymyr, et al.
        "Human-level control through deep reinforcement learning."
        Nature 518.7540 (2015): 529-533.
    :param observation_space:
    :param features_dim: Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 512):
        super(NatureCNN, self).__init__(observation_space, features_dim)
        # We assume CxHxW images (channels first)
        # Re-ordering will be done by pre-preprocessing or wrapper
        assert is_image_space(observation_space, check_channels=False), (
            "You should use NatureCNN "
            f"only with images not with {observation_space}\n"
            "(you are probably using `CnnPolicy` instead of `MlpPolicy` or `MultiInputPolicy`)\n"
            "If you are using a custom environment,\n"
            "please check it using our env checker:\n"
            "https://stable-baselines3.readthedocs.io/en/master/common/env_checker.html"
        )
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 4, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(8, 8, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(th.as_tensor(observation_space.sample()[None]).float()).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))

class MyActorCriticCnnPolicy(ActorCriticPolicy):
    """
    CNN policy class for actor-critic algorithms (has both policy and value prediction).
    Used by A2C, PPO and the likes.
    :param observation_space: Observation space
    :param action_space: Action space
    :param lr_schedule: Learning rate schedule (could be constant)
    :param net_arch: The specification of the policy and value networks.
    :param activation_fn: Activation function
    :param ortho_init: Whether to use or not orthogonal initialization
    :param use_sde: Whether to use State Dependent Exploration or not
    :param log_std_init: Initial value for the log standard deviation
    :param full_std: Whether to use (n_features x n_actions) parameters
        for the std instead of only (n_features,) when using gSDE
    :param sde_net_arch: Network architecture for extracting features
        when using gSDE. If None, the latent features from the policy will be used.
        Pass an empty list to use the states as features.
    :param use_expln: Use ``expln()`` function instead of ``exp()`` to ensure
        a positive standard deviation (cf paper). It allows to keep variance
        above zero and prevent it from growing too fast. In practice, ``exp()`` is usually enough.
    :param squash_output: Whether to squash the output using a tanh function,
        this allows to ensure boundaries when using gSDE.
    :param features_extractor_class: Features extractor to use.
    :param features_extractor_kwargs: Keyword arguments
        to pass to the features extractor.
    :param normalize_images: Whether to normalize images or not,
         dividing by 255.0 (True by default)
    :param optimizer_class: The optimizer to use,
        ``th.optim.Adam`` by default
    :param optimizer_kwargs: Additional keyword arguments,
        excluding the learning rate, to pass to the optimizer
    """

    def __init__(
        self,
        observation_space: gym.spaces.Space,
        action_space: gym.spaces.Space,
        lr_schedule: Schedule,
        net_arch: Optional[List[Union[int, Dict[str, List[int]]]]] = None,
        activation_fn: Type[nn.Module] = nn.Tanh,
        ortho_init: bool = True,
        use_sde: bool = False,
        log_std_init: float = 0.0,
        full_std: bool = True,
        sde_net_arch: Optional[List[int]] = None,
        use_expln: bool = False,
        squash_output: bool = False,
        features_extractor_class: Type[BaseFeaturesExtractor] = NatureCNN,
        features_extractor_kwargs: Optional[Dict[str, Any]] = None,
        normalize_images: bool = True,
        optimizer_class: Type[th.optim.Optimizer] = th.optim.Adam,
        optimizer_kwargs: Optional[Dict[str, Any]] = None,
    ):
        super(MyActorCriticCnnPolicy, self).__init__(
            observation_space,
            action_space,
            lr_schedule,
            net_arch,
            activation_fn,
            ortho_init,
            use_sde,
            log_std_init,
            full_std,
            sde_net_arch,
            use_expln,
            squash_output,
            features_extractor_class,
            features_extractor_kwargs,
            normalize_images,
            optimizer_class,
            optimizer_kwargs,
        )

In [142]:
import numpy as np
import gym
from gym import spaces
from puzzpy import PuzzTable
from stable_baselines3.common.env_checker import check_env

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class PuzzEnv(gym.Env):
  """
  パズドラの環境
  """
  # ColabのためGUIを実装できない
  metadata = {'render.modes': ['console']}

  def __init__(self):
    super(PuzzEnv, self).__init__()

    self.action_space = spaces.Discrete(5)

    # 状態はエージェントの座標になる
    # Discrete空間とBox空間の両方で表現できる
    # self.observation_space = spaces.Dict({'table': spaces.Box(low=0, high=5,
    #                                    shape=(5,6), dtype=np.float32),
    #                                    'start': spaces.Discrete(30),
    #                                    'turn': spaces.Box(low=1, high=155,
    #                                    shape=(1,), dtype=np.float32),
    #                                    'rew': spaces.Box(low=1, high=100,
    #                                    shape=(1,), dtype=np.float32)})
    
    #                                    'start': spaces.Box(low=0, high=1,
    #                                    shape=(5,6), dtype=np.float32),
    self.observation_space = spaces.Box(low=0, high=255, shape=(3,5,6), dtype=np.uint8)

  def zerosturn(self, table):
    zeros = np.zeros((5,6))
    zeros[0][0] = table.get_turn()
    zeros[0][1] = table.eval_otoshi()
    return zeros

  def retobs(self, table):

    
    # return {'table': np.array(table.get_table()).astype(np.float32),
    #           'start': np.array(table.get_XY_as_table()).astype(np.float32),
    #           'turn': table.get_turn(),'rew': table.eval_otoshi()}, table.eval_otoshi(), True, {}
    # return {'table': np.array(table.get_table()).astype(np.float32),
    #           'start': np.flatnonzero(table.get_XY_as_table())[0],
    #           'turn': table.get_turn(),'rew': table.eval_otoshi()}, table.eval_otoshi(), True, {}
    return np.stack([np.array(table.get_table()).astype(np.uint8),
                     np.array(table.get_XY_as_table()).astype(np.uint8),
                     self.zerosturn(table)]), table.eval_otoshi() - self.initreward, True, {}

  def reset(self):
    """
    【重要】観測はnumpy配列でなければならない
    :return: (np.array)
    """
    self.table = PuzzTable(155)
    self.initreward = self.table.eval_otoshi()
    return self.retobs(self.table)[0]

  def step(self, action):
    if action == 4:
      return self.retobs(self.table)

    next_table = self.table.next_tables()[action]

    if next_table.get_table()[0][0] == 127:
      return self.retobs(self.table)

    self.table = next_table

    if self.table.get_turn() <= 0:
      return self.retobs(self.table)

    # return {'table': np.array(self.table.get_table()).astype(np.float32),
    #           'start': np.array(self.table.get_XY_as_table()).astype(np.float32),
    #           'turn': self.table.get_turn(),'rew': self.table.eval_otoshi()}, 0, False, {}
    # return {'table': np.array(self.table.get_table()).astype(np.float32),
    #           'start': np.flatnonzero(table.get_XY_as_table())[0],
    #           'turn': self.table.get_turn(),'rew': self.table.eval_otoshi()}, 0, False, {}
    return np.stack([np.array(self.table.get_table()).astype(np.uint8),
                     np.array(self.table.get_XY_as_table()).astype(np.uint8),
                     self.zerosturn(self.table)]), 0, False, {}

  def render(self, mode='console', close=False):
    if mode != 'console':
      raise NotImplementedError()

    start = self.table.get_XY_as_table()
    table = self.table.get_table()
    for i in range(5):
      for j in range(6):
        if start[i][j] == 1:
          print(bcolors.FAIL + str(table[i][j]) + bcolors.ENDC, end='')
        else:
          print(table[i][j], end='')

      print('')
    # print('\n'.join([''.join(['{:2}'.format(item) for item in row]) 
    #   for row in self.table.get_table()]))
    # for i in self.table.get_table():
    #   print(*i)

check_env(PuzzEnv())

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/env_checker.py:48: UserWarning: The minimal resolution for an image is 36x36 for the default `CnnPolicy`. You might need to use a custom feature extractor cf. https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html
  "The minimal resolution for an image is 36x36 for the default `CnnPolicy`. "


In [143]:
# 手動での動作確認
env = PuzzEnv()

obs = env.reset()
env.render()

print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())

n_steps = 20
for step in range(n_steps):
  print("Step {}".format(step + 1))
  obs, reward, done, info = env.step(env.action_space.sample())
  print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render()
  if done:
    print("Goal reached!", "reward=", reward)
    break

266425
151664
534212
545342
111455
Box(0, 255, (3, 5, 6), uint8)
Discrete(5)
0
Step 1
obs= [[[  2.   6.   1.   4.   2.   5.]
  [  1.   5.   6.   6.   6.   4.]
  [  5.   3.   4.   2.   1.   2.]
  [  5.   4.   5.   3.   4.   2.]
  [  1.   1.   1.   4.   5.   5.]]

 [[  0.   0.   0.   0.   0.   0.]
  [  0.   0.   1.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.]]

 [[154.   2.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.]]] reward= 0 done= False
261425
156664
534212
545342
111455
Step 2
obs= [[[  2.   6.   1.   4.   2.   5.]
  [  1.   6.   5.   6.   6.   4.]
  [  5.   3.   4.   2.   1.   2.]
  [  5.   4.   5.   3.   4.   2.]
  [  1.   1.   1.   4.   5.   5.]]

 [[  0.   0.   0.   0.   0.   0.]
  [  0.   1.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.]
  [  0.   0.   0

In [144]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.envs.multi_input_envs import SimpleMultiObsEnv

# 環境の生成
env = PuzzEnv()

# 環境のラップ
env = Monitor(env, filename=None, allow_early_resets=True)
# env = DummyVecEnv([lambda: env])
# env = SimpleMultiObsEnv()

In [147]:
# エージェントの訓練
model = PPO(MyActorCriticCnnPolicy, env, verbose=1).learn(1600000)

ストリーミング出力は最後の 5000 行に切り捨てられました。
|    value_loss           | 0.212      |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.06        |
|    ep_rew_mean          | -0.03       |
| time/                   |             |
|    fps                  | 427         |
|    iterations           | 545         |
|    time_elapsed         | 2610        |
|    total_timesteps      | 1116160     |
| train/                  |             |
|    approx_kl            | 0.021555087 |
|    clip_fraction        | 0.132       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.48       |
|    explained_variance   | -0.0204     |
|    learning_rate        | 0.0003      |
|    loss                 | 0.07        |
|    n_updates            | 5440        |
|    policy_gradient_loss | -0.00322    |
|    value_loss           | 0.23        |
-----------------------------------------
----

In [1]:
# 訓練済みエージェントのテスト
obs = env.reset()
n_steps = 155
for step in range(n_steps):
  action, _ = model.predict(obs, deterministic=True)
  print("Step {}".format(step + 1))
  print("Action: ", action)
  obs, reward, done, info = env.step(action)
  # print('obs=', obs, 'reward=', reward, 'done=', done)
  print('score=', obs[2][0][1])
  env.render('console')
  if done:
    # VecEnvは、エピソード完了に遭遇すると自動的にリセットされることに注意
    print("Goal reached!", "reward=", reward)
    break

NameError: ignored